In [9]:
import tensorflow as tf
from tensorflow.keras import backend as k
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Dropout,Conv1D,Conv2D, MaxPooling1D,Flatten,BatchNormalization
from tensorflow.keras.callbacks import ModelCheckpoint, LearningRateScheduler
from sklearn.neighbors import KNeighborsClassifier
from sklearn.model_selection import KFold, train_test_split, GridSearchCV
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import confusion_matrix,accuracy_score, precision_score, recall_score, f1_score, classification_report
import matplotlib.pyplot as plt
import seaborn as sns
from collections import Counter
import numpy
import numpy as np
import pandas as pd
numpy.random.seed(2)

In [ ]:
# Load dataset for electrode 1 (E1)
dataset_e1= pd.read_csv("/content/drive/MyDrive/Code_IA_Test/E1.csv",delimiter=",")
dataset_e1=numpy.array(dataset_e1,float)
X_e1 = dataset_e1[:,1:7]
# Load dataset for electrode 2 (E2)
dataset_e2= pd.read_csv("/content/drive/MyDrive/Code_IA_Test/E2.csv",delimiter=",")
dataset_e2=numpy.array(dataset_e2,float)
X_e2 = dataset_e2[:,1:7]
# Extract output (Y)
Y= dataset_e1[:,8]
print(Y.shape)

In [ ]:
# Labels Histogram
Y_histogram = Counter(Y)
print("Labels Histogram:", Y_histogram)
plt.bar(Y_histogram.keys(), Y_histogram.values(), width=0.2)
plt.show()

In [ ]:
# Calculate the proportions of each class
interictal_percentage = np.sum(Y == 0) / len(Y)
preictal_percentage = np.sum(Y == 1) / len(Y)
ictal_percentage = np.sum(Y == 2) / len(Y)

# Print the average values of the Y column
print('Interictal: {0:.3f} | Preictal: {1:.3f} | Ictal: {2:.3f}'.format(interictal_percentage, preictal_percentage, ictal_percentage))

In [ ]:
# Step 1: Normalize data by dividing each column by the first column of the respective dataset
def normalize_data(X, dataset):
    lin, col = np.shape(X)
    for i in range(col - 2):
        X[:, i] = (X[:, i] / dataset[:, 0])

# Assuming dataset_e1 and dataset_e2 are defined

# Normalize data for X_e1 and X_e2
normalize_data(X_e1, dataset_e1)
normalize_data(X_e2, dataset_e2)

# Step 2: Use StandardScaler for further normalization
scaler1 = StandardScaler().fit(X_e1)
scaler2 = StandardScaler().fit(X_e2)
X_e1 = scaler1.transform(X_e1)
X_e2 = scaler2.transform(X_e2)

# Step 3: Combine normalized data into a 3D array
X = np.stack((X_e1, X_e2), axis=-1)

print("Shape of X_e1:", X_e1.shape)
print("Shape of X_e2:", X_e2.shape)
print("Shape of X:", X.shape)

# Step 4: Reshape the 3D array into a 2D array
nsamples, nfeatures, nchannels = X.shape
X_norm = X.reshape((nsamples, nfeatures * nchannels))

print("Shape of X_norm:", X_norm.shape)



In [ ]:
# Split the data into 80% training and 20% (testing and validating)
# Step 1: Split into 80% training and 20% (testing + validating)
X_train, X_val_test, Y_train, Y_val_test = train_test_split(X_norm, Y, test_size=0.2)

# Step 2: Further split the 20% into 10% validation and 10% testing
X_val, X_test, Y_val, Y_test = train_test_split(X_val_test, Y_val_test, test_size=0.5)

# Print shapes of the resulting datasets
print("Shape of X_train:", X_train.shape)
print("Shape of Y_train:", Y_train.shape)
print("Shape of X_val:", X_val.shape)
print("Shape of Y_val:", Y_val.shape)
print("Shape of X_test:", X_test.shape)
print("Shape of Y_test:", Y_test.shape)


In [ ]:
# Create a KNN model
knn_model = KNeighborsClassifier()

# Hyperparameter tuning using GridSearchCV
param_grid = {'n_neighbors': [3, 5, 7], 'weights': ['uniform', 'distance']}
grid_search = GridSearchCV(knn_model, param_grid, cv=5, scoring='accuracy')
grid_search.fit(X_train, Y_train)

# Get the best parameters from the grid search
best_params = grid_search.best_params_
print("Best Hyperparameters:", best_params)

# Train the KNN model with the best hyperparameters
best_knn_model = KNeighborsClassifier(n_neighbors=best_params['n_neighbors'], weights=best_params['weights'])
best_knn_model.fit(X_train, Y_train)

In [ ]:
# Make predictions on the test set
knn_predictions = best_knn_model.predict(X_test)

# Calculate various metrics
accuracy_knn = accuracy_score(knn_predictions, Y_test)
precision_knn = precision_score(knn_predictions, Y_test, average='macro')
recall_knn = recall_score(knn_predictions, Y_test, average='macro')
f1_knn = f1_score(knn_predictions, Y_test, average='macro')

# Print metric scores for the KNN model
print("\nMetric scores for KNN:")
print(" Accuracy: {:.2f} %".format(accuracy_knn * 100))
print(" Precision: {:.2f} %".format(precision_knn * 100))
print(" Recall: {:.2f} %".format(recall_knn * 100))
print(" F1: {:.2f} %".format(f1_knn * 100))

# Generate and print a detailed classification report for the KNN model
clf_report_knn = classification_report(Y_test, knn_predictions)
print("\nClassification Report for KNN:")
print(clf_report_knn)

# Display the confusion matrix
cm = confusion_matrix(Y_test, knn_predictions)
plt.figure(figsize=(8, 6))
sns.heatmap(cm, annot=True, fmt='d', cmap='Blues', xticklabels=['0', '1', '2'], yticklabels=['0', '1', '2'])
plt.xlabel('Predicted')
plt.ylabel('True')
plt.title('Confusion Matrix')
plt.show()